In [1]:
import advertools as adv
import adviz
import pandas as pd
pd.options.display.max_columns = None
import plotly.express as px
from dash import Dash, Input, Output, State, dcc, html
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
load_figure_template(['slate'])

In [2]:

df = pd.read_csv('2023-10-07-2024-01-23-Middle_East.csv')
df['notes_clean'] = ['<br>'.join(adv.ad_from_string(note, [70 for i in range(20)])) for note in df['notes']]
df['notes_clean'] = df['notes_clean'].str.replace('(<br>)+$', '', regex=True)
df['event_date_clean'] = pd.to_datetime(df['event_date'])

In [3]:
# df.head(2)

In [4]:
df['week'] = [pd.Period(date, freq='W') for date in df['event_date_clean']]

In [5]:
weekly = pd.pivot_table(df[df['country'].isin(['Palestine', 'Israel'])], index=['week', 'country', 'latitude', 'longitude'], values='fatalities', aggfunc='sum').reset_index().query('fatalities > 0')

In [6]:
daily_fatalities = df[df['country'].isin(['Palestine', 'Israel'])].groupby(['event_date_clean', 'country'], as_index=False).agg({'fatalities': 'sum'})

#### By [@EliasDabbas](https://x.com/eliasdabbas)

#### Data: The Armed Conflict Location & Event Data Project (ACLED) [acleddata.com](https://acleddata.com)

#### Date range: (Oct 7, 2023 - Jan 19, 2024)

# Total deaths and ratio

In [7]:
total_fatalities = daily_fatalities.groupby('country')['fatalities'].sum().reset_index()

In [8]:
import warnings
warnings.filterwarnings(category=FutureWarning, action='ignore')


hashtag = '<a href="https://twitter.com/hashtag/StrikeForGaza">#StrikeForGaza</a><br>'

In [9]:
ratio = total_fatalities['fatalities'][1] / total_fatalities['fatalities'][0]
fig = px.bar(
    x=total_fatalities['country'],
    y=total_fatalities['fatalities'],
    color=daily_fatalities.groupby('country')['fatalities'].sum().index,
    labels={'x': '', 'y': 'count', 'color': ''},
    # text=[adviz.flag('il'), adviz.flag('ps')],
    text=total_fatalities['fatalities'],
    height=600, width=900,
    title=f'Total deaths: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}<br>Ratio: {ratio:.1f}:1'
)
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.15, xref='paper', yref='paper', showarrow=False, font={'size': 20})
fig.layout.margin.b = 100
fig.layout.margin.t = 130

fig

## Daily cumulative deaths

In [10]:
import plotly.graph_objects as go

ps_daily_cumsum = daily_fatalities.query('country=="Palestine"').assign(cum_fatalities=lambda df: df['fatalities'].cumsum())
il_daily_cumsum = daily_fatalities.query('country=="Israel"').assign(cum_fatalities=lambda df: df['fatalities'].cumsum())

fig = go.Figure()

fig.add_scatter(
    x=il_daily_cumsum['event_date_clean'],
    y=il_daily_cumsum['cum_fatalities'],
    mode='markers+lines',
    marker={'size': 10},
    name='Israel')
fig.add_scatter(
    x=ps_daily_cumsum['event_date_clean'],
    y=ps_daily_cumsum['cum_fatalities'],
    mode='markers+lines',
    marker={'size': 10},
    name='Palestine')

fig.layout.title = f'Cumulative deaths {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}'
fig.layout.title.font.size = 20
fig.layout.height = 650
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.15, xref='paper', yref='paper', showarrow=False, font={'size': 20})
fig.layout.margin.b = 100
fig.layout.hovermode = 'x unified'
fig.write_html('cumulative_daily_deaths.html', config={'displayModeBar': False})
fig

## Weekly deaths map

In [11]:
df_subset = df[df['country'].isin(['Palestine', 'Israel'])].sort_values(['event_date_clean'])
df_subset = weekly
fig = px.scatter_geo(
    df_subset,
    lat='latitude',
    lon='longitude',
    height=750,
    title='Weekly deaths map - Palestine & Israel',
    size='fatalities',
    size_max=20,
    hover_name='country',
    color_discrete_sequence=px.colors.qualitative.D3,
    color='country',
    template='slate',
    animation_frame='week',
)
fig.layout.geo.showframe = False
fig.layout.geo.showcountries = True
fig.layout.geo.projection.type = 'natural earth'
fig.layout.geo.lataxis.range = df_subset['latitude'].min() * 0.98, df_subset['latitude'].max() * 1.01
fig.layout.geo.lonaxis.range = df_subset['longitude'].min() * 0.98, df_subset['longitude'].max() * 1.01

# fig.layout.geo.landcolor = 'white'

# fig.layout.geo.bgcolor = '#E5ECF6'
# fig.layout.paper_bgcolor = '#E5ECF6'
fig.layout.geo.countrycolor = 'gray'
fig.layout.geo.coastlinecolor = 'gray'
fig.layout.geo.resolution = 50
for data in fig.data:
    data.hovertemplate = data.hovertemplate.replace('=', ': ')

fig.layout.showlegend = False
fig
# fig.write_html('daily_fatalities_map.html', auto_play=False)
# !open daily_fatalities_map.html

## Monthly deaths

In [12]:
daily_fatalities['month'] = [pd.Period(x, 'M') for x in  daily_fatalities['event_date_clean']]
daily_fatalities['week'] = [pd.Period(x, 'W') for x in  daily_fatalities['event_date_clean']]

In [13]:
monthly_fatalities = daily_fatalities.groupby(['month', 'country'])['fatalities'].sum().reset_index()
weekly_fatalities = daily_fatalities.groupby(['week', 'country'])['fatalities'].sum().reset_index()

In [14]:
fig = px.bar(
    monthly_fatalities,
    x=monthly_fatalities['month'].astype(str),
    y='fatalities',
    labels={'fatalities': 'count', 'event_date_clean': ''},
    height=650,
    title=f'Monthly deaths {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    color='country',
    facet_col='country')
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = fig.layout.annotations[i].text.replace('country=', '')
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.2, xref='paper', yref='paper', showarrow=False, font={'size': 20})
fig.layout.margin.b = 100
fig.layout.margin.t = 150
fig.layout.font.size =+ 14
fig

## Weekly Deaths

In [15]:
fig = px.bar(
    weekly_fatalities,
    x=weekly_fatalities['week'].astype(str),
    y='fatalities',
    labels={'fatalities': 'count', 'event_date_clean': '', 'x': ''},
    height=650,
    title=f'Weekly deaths {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    color='country',
    facet_col='country')
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = fig.layout.annotations[i].text.replace('country=', '')
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.35, xref='paper', yref='paper', showarrow=False, font={'size': 20})
fig.layout.margin.b = 100
fig.layout.margin.t = 150
fig.layout.font.size =+ 14
fig

## Daily Deaths

In [16]:
fig = px.bar(
    daily_fatalities,
    x='event_date_clean',
    y='fatalities',
    color='country',
    labels={'fatalities': 'count', 'event_date_clean': ''},
    height=650,
    title=f'Daily deaths {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    facet_col='country')
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = fig.layout.annotations[i].text.replace('country=', '')
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.2, xref='paper', yref='paper', showarrow=False, font={'size': 20})
fig.layout.margin.b = 100
fig.layout.margin.t = 150
fig.layout.font.size =+ 14
fig

In [17]:
# Deaths of Dec 5, 2023
# df[df['country'].eq('Palestine') & df['event_date_clean'].astype(str).eq('2023-12-05')][['admin2','fatalities']].groupby('admin2').sum().sort_values('fatalities', ascending=False)

In [18]:
# df[df['country'].eq('Palestine') & df['event_date_clean'].astype(str).eq('2023-12-05') & df['fatalities'].gt(0)][['fatalities', 'event_type', 'notes', 'admin2']].sort_values('fatalities', ascending=False)

In [19]:
total_fatalities = daily_fatalities.groupby('country')['fatalities'].sum().reset_index()

## Deaths by city – Palestine

In [20]:
fig = adviz.style_table(
    df
    .groupby(['country', 'admin1', 'admin2'],
             as_index=False)
    .agg({'fatalities': 'sum'})
    .query('country=="Palestine"')
    .sort_values('fatalities', ascending=False),
    column_types=['text', 'category', 'text', 'bar'],
    title=f'Total deaths per city: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    column_widths=[0.1, 0.2, 0.19, 0.55],
    height=650, width=1000, theme='slate',
)
fig.data[3].texttemplate = '%{text:,}'
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.1, xref='paper', yref='paper', showarrow=False, font={'size': 20})

fig

## Deaths by city – Israel

In [21]:
fig = adviz.style_table(
    df
    .groupby(['country', 'admin1', 'admin2', 'admin3'],
             as_index=False)
    .agg({'fatalities': 'sum'})
    .query('country=="Israel"')
    .sort_values('fatalities', ascending=False).head(10),
    column_types=['text', 'category', 'category', 'text', 'bar'],
    title=f'Total deaths per city: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    column_widths=[0.1, 0.2, 0.2, 0.2, 0.4],
    height=500, 
    width=1000, theme='slate',
)
fig.data[4].texttemplate = '%{text:,}'
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.18, xref='paper', yref='paper', showarrow=False, font={'size': 20})

fig

In [22]:
by_country = df[df['country'].isin(['Palestine', 'Israel'])]['country'].value_counts().reset_index()
by_country['flag'] = adviz.flag('ps'), adviz.flag('il')

In [23]:
event_types = df[['country', 'disorder_type', 'event_type', 'sub_event_type']].value_counts().reset_index()

In [24]:
# fig = adviz.style_table(
#     event_types[event_types['country'].eq('Israel')].iloc[:, 1:],
#     column_types=['category', 'category', 'text', 'bar'],
#     column_widths=[0.2, 0.2, 0.25, 0.4],
#     theme='slate',
#     title=f'<a href="https://twitter.com/hashtag/StrikeForGaza">#StrikeForGaza</a><br>Disorder & event types - Israel: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
#     height=700, width=1300)
# fig.data[3].texttemplate = '%{text:,}'
# for annotation in fig.layout.annotations:
#     annotation.text = annotation.text.replace('_', ' ')
# fig.add_annotation(
#     text='Data: ACLED acleddata.com',
#     x=-0, y=-0.1, xref='paper', yref='paper', showarrow=False, font={'size': 20})

# fig.layout.margin.t = 150
# fig

In [25]:
# fig = adviz.style_table(
#     event_types[event_types['country'].eq('Palestine')].iloc[:, 1:],
#     column_types=['category', 'category', 'text', 'bar'],
#     column_widths=[0.2, 0.2, 0.25, 0.4],
#     theme='slate',
#     title=f'<a href="https://twitter.com/hashtag/StrikeForGaza">#StrikeForGaza</a><br>Disorder & event types - Palestine: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
#     height=700, width=1300)
# fig.data[3].texttemplate = '%{text:,}'
# for annotation in fig.layout.annotations:
#     annotation.text = annotation.text.replace('_', ' ')
# fig.add_annotation(
#     text='Data: ACLED acleddata.com',
#     x=-0, y=-0.1, xref='paper', yref='paper', showarrow=False, font={'size': 20})

# fig.layout.margin.t = 150
# fig

In [26]:
ps_top_cities = df[df['country'].eq('Palestine')].groupby('admin2')['fatalities'].sum().sort_values(ascending=False).head(5).index
il_top_cities = df[df['country'].eq('Israel')].groupby('admin2')['fatalities'].sum().sort_values(ascending=False).head(5).index

In [27]:
ps_daily_city = (df
 [df['country'].eq('Palestine') & df['admin2'].isin(ps_top_cities)]
 .groupby(['event_date_clean', 'admin1', 'admin2'], as_index=False)
 .agg({'fatalities': 'sum'}))
il_daily_city = (df
 [df['country'].eq('Israel') & df['admin2'].isin(il_top_cities)]
 .groupby(['event_date_clean', 'admin1', 'admin2'], as_index=False)
 .agg({'fatalities': 'sum'}))


## Daily deaths by city – Palestine

In [28]:
fig = px.bar(
    ps_daily_city,
    x='event_date_clean',
    y='fatalities',
    color='admin2',
    labels={'fatalities': 'daily deaths'},
    title=f'Daily deaths per city - Palestine: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    facet_col='admin2',
    height=650)
for annotation in fig.layout.annotations:
    annotation.text = annotation.text.replace('admin2=', '')
fig.update_xaxes(title_text='')
fig.update_annotations(font_size=17)
fig.update_legends(orientation='h', x=0.5, title='', xanchor='center')
fig.layout.margin.r = 10
fig.layout.margin.t = 150
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.15, xref='paper', yref='paper', showarrow=False, font={'size': 20})

fig

## Daily deaths by city – Israel

In [29]:
fig = px.bar(
    il_daily_city,
    x='event_date_clean',
    y='fatalities',
    color='admin2',
    labels={'fatalities': 'daily deaths'},
    title=f'Daily deaths per city - Israel: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    facet_col='admin2',
    height=650)
for annotation in fig.layout.annotations:
    annotation.text = annotation.text.replace('admin2=', '')
fig.update_xaxes(title_text='')
fig.update_annotations(font_size=17)
fig.update_legends(orientation='h', x=0.5, title='', xanchor='center')
fig.layout.margin.r = 10
fig.layout.margin.t = 150
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.15, xref='paper', yref='paper', showarrow=False, font={'size': 20})

fig

In [30]:
types_counts = df[['country', 'disorder_type', 'event_type', 'sub_event_type']].value_counts().reset_index()
# types_counts.head()

In [31]:
top_events = types_counts.sort_values('count', ascending=False)['sub_event_type'].drop_duplicates().head(9)

## Event types and counts

In [32]:
fig = px.bar(
    types_counts[types_counts['sub_event_type'].isin(top_events) & types_counts['country'].isin(['Palestine', 'Israel'])],
    x='country',
    y='count',
    color='event_type',
    title=f'Event types and counts: {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}',
    facet_col='sub_event_type',
    labels={'country': ''},
    color_discrete_sequence=px.colors.qualitative.Bold,
    height=500)
fig.layout.legend.title.text = "Event type"
fig.layout.legend.orientation = 'h'
for annotation in fig.layout.annotations:
    annotation.text = annotation.text.replace('sub_event_type=', '').replace(' ', '<br>')
fig.layout.margin.r = 20
fig.layout.margin.t = 170
fig.layout.margin.b = 400
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.75, xref='paper', yref='paper', showarrow=False, font={'size': 20})
fig.layout.margin.b = 100
fig.layout.legend.y = -0.15

fig

In [33]:
air_strikes = (df
               [df['sub_event_type'].eq('Air/drone strike')]
               .groupby(['event_date_clean', 'country'])
               .agg({'event_id_cnty': 'count', 'fatalities': 'sum'})
               .reset_index()
               .assign(rate=lambda df: df['fatalities'].div(df['event_id_cnty'])))
# air_strikes.head()

## Daily air/drone strikes and resulting deaths - Palestine

In [34]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1)

fig.add_bar(
    x=air_strikes.query('country=="Palestine"')['event_date_clean'],
    y=air_strikes.query('country=="Palestine"')['event_id_cnty'],
    row=1, col=1,
    name='Air/drone strikes')


fig.add_bar(
    x=air_strikes.query('country=="Palestine"')['event_date_clean'],
    y=air_strikes.query('country=="Palestine"')['fatalities'],
    row=2, col=1,
    name='Deaths',
)



fig.layout.hovermode = 'x unified'
fig.layout.height = 650
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.15, xref='paper', yref='paper', showarrow=False, font={'size': 20})

fig.layout.title = f'Air/drone strikes - Daily (Palestine): {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}'
fig

## Daily air/drone strikes and resulting deaths - Israel

In [35]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1)

fig.add_bar(
    x=air_strikes.query('country=="Israel"')['event_date_clean'],
    y=air_strikes.query('country=="Israel"')['event_id_cnty'],
    row=1, col=1,
    name='Air/drone strikes')


fig.add_bar(
    x=air_strikes.query('country=="Israel"')['event_date_clean'],
    y=air_strikes.query('country=="Israel"')['fatalities'],
    row=2, col=1,
    name='Deaths',
)



fig.layout.hovermode = 'x unified'
fig.layout.height = 650
fig.add_annotation(
    text='Data: ACLED acleddata.com',
    x=-0, y=-0.15, xref='paper', yref='paper', showarrow=False, font={'size': 20})

fig.layout.title = f'Aid/drone strikes - Daily (Israel): {str(df["event_date_clean"].min())[:-9]} – {str(df["event_date_clean"].max())[:-9]}'
fig

In [36]:
# df[df['event_date_clean'].eq('2023-12-05') &
#    df['country'].eq('Palestine') &
#    df['fatalities'].gt(0)][['fatalities','notes', 'admin2']]

In [37]:
# adviz.value_counts_plus(df[df['country'].eq('Israel')]['source'].str.split('; ').explode())

In [38]:
# adviz.value_counts_plus(df[df['country'].eq('Palestine')]['source'].str.split('; ').explode())

In [39]:
# adviz.value_counts_plus(df['country'], show_top=15)

In [40]:
# adviz.value_counts_plus(pd.concat([df['actor1'], df['actor2']]), show_top=20, dropna=True)

In [41]:
# df2 = df[df['event_type'].isin(['Protests'])]
# daily_protests = df2.groupby(['event_date_clean', 'country'], as_index=False)['event_id_cnty'].count()
# daily_protests.head()